# Selenium - 동적 웹페이지 스크래핑

In [1]:
# # Seleium 드라이버 생성
# from selenium import webdriver
# from selenium.webdriver.chrome.service import Service
# from webdriver_manager.chrome import ChromeDriverManager

# # Chrome 옵션 설정
# options = webdriver.ChromeOptions()
# # options.add_argument('--headless')  # 화면없이 실행

# # 드라이버 서비스 생성
# service = Service(ChromeDriverManager().install())

# # 웹 드라이버 초기화
# driver = webdriver.Chrome(service=service, options=options)


from selenium import webdriver
from selenium.webdriver.chrome.service import Service

# Chrome 옵션 설정

options = webdriver.ChromeOptions()
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

# Selenium Manager를 사용하여 자동으로 드라이버 관리
service = Service()

# 웹 드라이버 초기화
driver = webdriver.Chrome(service=service, options=options)

## 뉴스 서비스에 접속

In [2]:
# 뉴스 사이트
url = "https://news.daum.net/"

driver.get(url)

In [3]:
driver.page_source

'<html lang="ko" class="os_mac chrome pc version_128_0_0_0 "><head>\n<meta charset="utf-8">\n<meta name="referrer" content="always">\n\n<meta property="og:author" content="Daum 뉴스">\n<meta property="og:site_name" content="다음뉴스">\n<meta property="og:title" content="다음뉴스 | 홈">\n<meta property="og:image" content="https://t1.daumcdn.net/media/img-media/mobile/meta/news.png">\n<meta property="og:description" content="다음뉴스">\n<meta property="og:url" content="https://news.daum.net/">\n<link rel="shortcut icon" href="https://m2.daumcdn.net/img-media/2010ci/Daum_favicon.ico">\n\n<title>다음뉴스 | 홈</title>\n\n<meta http-equiv="X-UA-Compatible" content="IE=edge">\n\n<link rel="stylesheet" type="text/css" href="//t1.daumcdn.net/media/kraken/news/c66925f/style.css.merged.css">\n<link rel="stylesheet" type="text/css" href="//t1.daumcdn.net/media/kraken/news/c66925f/calendar.css.merged.css">\n\n<!--[if lte IE 8]>\n<script src="https://m2.daumcdn.net/svc/original/U0301/cssjs/JSON-js/fc535e9cc8/json2.min.

In [4]:
# 페이지 소스 가져오기
from bs4 import BeautifulSoup
page_source = driver.page_source

soup = BeautifulSoup(page_source, 'html.parser')

# 태그 검색
print('title 태그 요소: ', soup.title)
print('title 태그 이름: ', soup.title.name)
print('title 태그 문자열: ', soup.title.text)

title 태그 요소:  <title>다음뉴스 | 홈</title>
title 태그 이름:  title
title 태그 문자열:  다음뉴스 | 홈


## 경제 카테고리를 선택

In [5]:
from selenium.webdriver.common.by import By

# 경제 뉴스 클릭
economy = driver.find_element(By.CSS_SELECTOR, '#gnbContent > div > ul > li:nth-child(4) > a > span')

# #gnbContent > div > ul > li:nth-child(4) > a > span
economy

<selenium.webdriver.remote.webelement.WebElement (session="52f153656f60dcff8aa74479856267b1", element="f.25C2554947C054220FA55F07A43D0D47.d.0AB1339218CD137E23051CCFF28B6680.e.35")>

In [6]:
economy.click()

driver.implicitly_wait(5) # 5초 대기


In [7]:
# 페이지 소스 가져오기

soup = BeautifulSoup(driver.page_source, 'html.parser')

# 실시간 경제 뉴스 검색
news = soup.select('#timeline > ul > li')
len(news)

20

In [8]:
news[0]

<li>
<em class="txt_time">15:34</em>
<strong class="tit_g">
<span class="info_cp">파이낸셜뉴스</span>
<a class="link_txt" data-tiara-custom="contentUniqueKey=hamny-20240828153432013" data-tiara-id="20240828153432013" data-tiara-layer="article" data-tiara-ordnum="1" data-tiara-type="harmony" href="https://v.daum.net/v/20240828153432013">'물량관리 동참' 하나은행, 다주택자 생활안정자금 한도 1억원으로 축소</a>
</strong>
</li>

In [9]:
news[0].select("a.link_txt")[0].text.strip()

"'물량관리 동참' 하나은행, 다주택자 생활안정자금 한도 1억원으로 축소"

In [10]:
news[0].select_one("a.link_txt").text.strip()

"'물량관리 동참' 하나은행, 다주택자 생활안정자금 한도 1억원으로 축소"

In [11]:
# 데이터 추출하여 정리
data = {'title':[], 'agency':[], 'category':[], 'link':[]}

for item in news:
    title = item.select_one('strong > a').text
    agency = item.select_one('span.info_cp').text
    category = '경제'
    link = item.select_one('strong > a')['href']
    
    data['title'].append(title)
    data['agency'].append(agency)
    data['category'].append(category)
    data['link'].append(link)


# 데이터 프레임 생성
import pandas as pd
df = pd.DataFrame(data)
df.head()

,title,agency,category,link
0,"'물량관리 동참' 하나은행, 다주택자 생활안정자금 한도 1억원으로 축소",파이낸셜뉴스,경제,https://v.daum.net/v/20240828153432013
1,"제네시스 '하이브리드' 나온다…현대차, 10년간 120조 투자",한국경제,경제,https://v.daum.net/v/20240828153401981
2,"임종룡 ""조사·수사 결과에 맞는 조치와 절차 겸허히 따를 것""",중앙일보,경제,https://v.daum.net/v/20240828153316944
3,실제 AI 쓰는 기업은 10곳 중 3곳…제조업은 25% 미만,매일경제,경제,https://v.daum.net/v/20240828153301925
4,산업 전반 DX 가속화…“작업자→디지털관리자 경력 재설계해야”,전자신문,경제,https://v.daum.net/v/20240828153246908


In [12]:
len(df)

20

## 페이지 이동하면서 반복적으로 수집

In [20]:
# 페이지네이션 배열
#timeline > div > div > a

page_numbers = driver.find_elements(By.CSS_SELECTOR, '#timeline > div > div > a')
len(page_numbers)
#timeline > div > div > a:nth-child(3)

10

In [21]:
page_numbers[0]

<selenium.webdriver.remote.webelement.WebElement (session="52f153656f60dcff8aa74479856267b1", element="f.25C2554947C054220FA55F07A43D0D47.d.7FFCA705C45E3EEC6B8842F419E3E463.e.77")>

In [22]:
for p in page_numbers:
    print(p.text)

1
3
4
5
6
7
8
9
10
다음


In [16]:
current_page = 1

In [23]:
for p in page_numbers:
    print(p.text)
    if (p.text != '다음') and (int(p.text) == current_page + 1):
        p.click()
        driver.implicitly_wait(5)
        current_page = current_page + 1
        print(p.text, current_page)
        break

1
3
3 3


In [24]:
current_page

3

In [25]:
# 페이지 소스 가져오기
soup = BeautifulSoup(driver.page_source, 'html.parser')

# 실시간 경제 뉴스 검색
news = soup.select('#timeline > ul > li')
len(news)

20

In [26]:
# 뉴스 추출하여 기존 데이터에 추가
for item in news:
    title = item.select_one('strong > a').text
    agency = item.select_one('span.info_cp').text
    category = '경제'
    link = item.select_one('strong > a')['href']
    
    data['title'].append(title)
    data['agency'].append(agency)
    data['category'].append(category)
    data['link'].append(link)


# 데이터 프레임 생성
df = pd.DataFrame(data)
print(len(df))

df.head()

40


,title,agency,category,link
0,"'물량관리 동참' 하나은행, 다주택자 생활안정자금 한도 1억원으로 축소",파이낸셜뉴스,경제,https://v.daum.net/v/20240828153432013
1,"제네시스 '하이브리드' 나온다…현대차, 10년간 120조 투자",한국경제,경제,https://v.daum.net/v/20240828153401981
2,"임종룡 ""조사·수사 결과에 맞는 조치와 절차 겸허히 따를 것""",중앙일보,경제,https://v.daum.net/v/20240828153316944
3,실제 AI 쓰는 기업은 10곳 중 3곳…제조업은 25% 미만,매일경제,경제,https://v.daum.net/v/20240828153301925
4,산업 전반 DX 가속화…“작업자→디지털관리자 경력 재설계해야”,전자신문,경제,https://v.daum.net/v/20240828153246908


In [ ]:
# 드라이버 종료
driver.close()

In [27]:
# 페이지를 이동하면서 데이터를 수집하여 정리하는 함수
def get_news(driver, category):

    data = {'title':[], 'agency':[], 'category':[], 'link':[]}

    current_page = 1
    
    while current_page < 6:
        try:
            soup = BeautifulSoup(driver.page_source, 'html.parser')
            news = soup.select('#timeline > ul > li')
            
            for item in news:
                title = item.select_one('strong > a').text
                agency = item.select_one('span.info_cp').text
                link = item.select_one('strong > a')['href']
                
                data['title'].append(title)
                data['agency'].append(agency)
                data['category'].append(category)
                data['link'].append(link)
            
            page_numbers = driver.find_elements(By.CSS_SELECTOR, '#timeline > div > div > a')
            driver.implicitly_wait(5)
            
            for p in page_numbers:
                if (p.text != '다음') and (int(p.text) == current_page + 1):
                    p.click()
                    driver.implicitly_wait(5)
                    current_page = current_page + 1
                    print(p.text, current_page)
                    break        

        except:
            pass
    
    df = pd.DataFrame(data)
    
    driver.close()

    return df

In [36]:
# 웹 드라이버 초기화
driver = webdriver.Chrome(service=service, options=options)

In [37]:
# 뉴스 사이트 접속
url = "https://news.daum.net/"
driver.get(url)

In [38]:
# 뉴스 카테고리 (경제 등) 우클릭 > 검사 > 하이라이트 된 요소 선택 > 복사 > 선택자 복사
# gnbContent > div > ul > li.on > a > span
# li 까지만 사용해서 보면 다음과 같음

In [39]:
# 뉴스 카테고리 메뉴 배열
category_tabs = driver.find_elements(By.CSS_SELECTOR, '#gnbContent > div > ul > li')
category_tabs


[<selenium.webdriver.remote.webelement.WebElement (session="e24bb6d68a292bb51469e351d7c0044f", element="f.D1FB91FA74C98971F84BAA7ACB1CBC49.d.66FA7BB5E3FF7181375E45EAD3152C93.e.34")>,
 <selenium.webdriver.remote.webelement.WebElement (session="e24bb6d68a292bb51469e351d7c0044f", element="f.D1FB91FA74C98971F84BAA7ACB1CBC49.d.66FA7BB5E3FF7181375E45EAD3152C93.e.35")>,
 <selenium.webdriver.remote.webelement.WebElement (session="e24bb6d68a292bb51469e351d7c0044f", element="f.D1FB91FA74C98971F84BAA7ACB1CBC49.d.66FA7BB5E3FF7181375E45EAD3152C93.e.36")>,
 <selenium.webdriver.remote.webelement.WebElement (session="e24bb6d68a292bb51469e351d7c0044f", element="f.D1FB91FA74C98971F84BAA7ACB1CBC49.d.66FA7BB5E3FF7181375E45EAD3152C93.e.37")>,
 <selenium.webdriver.remote.webelement.WebElement (session="e24bb6d68a292bb51469e351d7c0044f", element="f.D1FB91FA74C98971F84BAA7ACB1CBC49.d.66FA7BB5E3FF7181375E45EAD3152C93.e.38")>,
 <selenium.webdriver.remote.webelement.WebElement (session="e24bb6d68a292bb51469e351d

In [40]:
category_tabs[3].text

'경제'

In [41]:
category_dict = {item.text:item for item in category_tabs}
category_dict

# item.text : key
# item : value


{'홈': <selenium.webdriver.remote.webelement.WebElement (session="e24bb6d68a292bb51469e351d7c0044f", element="f.D1FB91FA74C98971F84BAA7ACB1CBC49.d.66FA7BB5E3FF7181375E45EAD3152C93.e.34")>,
 '사회': <selenium.webdriver.remote.webelement.WebElement (session="e24bb6d68a292bb51469e351d7c0044f", element="f.D1FB91FA74C98971F84BAA7ACB1CBC49.d.66FA7BB5E3FF7181375E45EAD3152C93.e.35")>,
 '정치': <selenium.webdriver.remote.webelement.WebElement (session="e24bb6d68a292bb51469e351d7c0044f", element="f.D1FB91FA74C98971F84BAA7ACB1CBC49.d.66FA7BB5E3FF7181375E45EAD3152C93.e.36")>,
 '경제': <selenium.webdriver.remote.webelement.WebElement (session="e24bb6d68a292bb51469e351d7c0044f", element="f.D1FB91FA74C98971F84BAA7ACB1CBC49.d.66FA7BB5E3FF7181375E45EAD3152C93.e.37")>,
 '국제': <selenium.webdriver.remote.webelement.WebElement (session="e24bb6d68a292bb51469e351d7c0044f", element="f.D1FB91FA74C98971F84BAA7ACB1CBC49.d.66FA7BB5E3FF7181375E45EAD3152C93.e.38")>,
 '문화': <selenium.webdriver.remote.webelement.WebElement 

In [42]:
# 경제 뉴스 클릭
category_dict['경제'].click()
driver.implicitly_wait(5)

In [43]:
# 데이터 수집
df_economy = get_news(driver, '경제')
print(len(df_economy))

df_economy.head()

2 2
3 3
4 4
5 5
6 6
160


,title,agency,category,link
0,"전기차 생각만큼 안 팔려도… 현대차 ""美 조지아 공장 문제없다""",데일리안,경제,https://v.daum.net/v/20240828200914773
1,"버크셔해서웨이, 시가총액 1조달러 클럽 도전",한국경제,경제,https://v.daum.net/v/20240828200910772
2,29일 오전 6시…엔비디아發 `태풍`이 온다,디지털타임스,경제,https://v.daum.net/v/20240828200232564
3,"애플·구글의 앱마켓 독점 견제가 기회...원스토어 ""지역 맞춤형 전략으로 해외 간다""",한국일보,경제,https://v.daum.net/v/20240828200231561
4,[여기는 전남] 순천 시내버스 노선 개편…중복노선 줄이고 읍면 지선 체계,KBS,경제,https://v.daum.net/v/20240828200049520


In [44]:
# 페이지 이동할 때 동적으로 기사 배열이 바뀌면서 일부 뉴스가 중복 수집되는 현상이 있습니다. 중복을 제거합니다. (!!!)
df_economy = df_economy.drop_duplicates()
len(df_economy)

80

In [45]:
# 데이터 저장
df_economy.to_csv('daum_economy_news.csv', index=False)